In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
import sys
import re

In [2]:
model = keras.models.load_model('saved_model.h5', compile = False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
df = pd.DataFrame(columns=['D0:4D:C6:02:E5:C0', 'D0:4D:C6:02:95:61', 'D0:4D:C6:02:95:70',
       'D0:4D:C6:02:47:F0', 'D0:4D:C6:02:47:E0', 'D0:4D:C6:02:C2:81',
       'D0:4D:C6:01:E6:20', 'D0:4D:C6:02:C2:80', 'D0:4D:C6:01:E6:21',
       'D0:4D:C6:02:7E:50', 'D0:D3:E0:B1:3A:60', 'D0:4D:C6:02:7E:51',
       'D0:4D:C6:02:7E:41', '00:4E:35:CF:97:A1', 'D0:4D:C6:03:4D:51',
       'D0:4D:C6:02:7E:40', 'D0:4D:C6:03:4D:41', 'D0:4D:C6:02:E5:C1',
       '00:4E:35:CF:97:A0', 'D0:D3:E0:B1:3A:61', '00:4E:35:CF:2E:51',
       'D0:4D:C6:02:47:F1', '00:4E:35:CF:2E:41', 'D0:4D:C6:02:95:60',
       'D0:4D:C6:03:4D:40', 'D0:4D:C6:03:4D:50', 'D0:4D:C6:02:E5:D1',
       'D0:4D:C6:02:95:71', '40:E3:D6:24:25:23', '00:4E:35:CF:2E:40',
       '40:E3:D6:24:25:22'])

In [4]:
def make_prediction(data, df):
    result = re.findall(r"Address[\S\s]{265}", data)

    csuAPs = []
    for item in result:
        if "csu-net" in item or "csu-visitor" in item:
            csuAPs.append(item)

    qualities = {}
    for ap in csuAPs:
        mac = re.findall(r"[\S\s]{2}:[\S\s]{2}:[\S\s]{2}:[\S\s]{2}:[\S\s]{2}:[\S\s]{2}", ap)[0]
        quality = re.findall(r"\d\d/\d\d", ap)[0]
        qualityInt = int(quality[0:2])
        qualities[mac] = quality

    dictionary = {}
    for column in df.columns:
        dictionary[column] = qualities[column].split('/')[0] if column in qualities else "0"

    df = df.append(dictionary, ignore_index = True)
    prediction = model.predict(df.astype(int).to_numpy() / 70)
    return prediction

In [5]:
dftest = df.iloc[:,:].to_numpy() / 70

In [7]:
!iwlist wlan0 scan

wlan0     Scan completed :
          Cell 01 - Address: 0A:A3:E2:62:79:78
                    Channel:9
                    Frequency:2.452 GHz (Channel 9)
                    Quality=70/70  Signal level=-38 dBm  
                    Encryption key:on
                    ESSID:"FBI2"
                    Bit Rates:1 Mb/s; 2 Mb/s; 5.5 Mb/s; 11 Mb/s
                    Bit Rates:6 Mb/s; 9 Mb/s; 12 Mb/s; 18 Mb/s; 24 Mb/s
                              36 Mb/s; 48 Mb/s; 54 Mb/s
                    Mode:Master
                    Extra:tsf=0000000575db0077
                    Extra: Last beacon: 1012ms ago
                    IE: Unknown: 000446424932
                    IE: Unknown: 010482848B96
                    IE: Unknown: 030109
                    IE: Unknown: 3B110C010204050C1617181A1B1C1D1E202180
                    IE: Unknown: 2A0100
                    IE: Unknown: 32080C1218243048606C
                    IE: IEEE 802.11i/WPA2 Version 1
                        Group Cipher : CCMP

In [8]:
import subprocess
from subprocess import PIPE
import time

In [9]:
from jetbot import Robot
robot = Robot()
robot.right(0.3)
time.sleep(0.25)
robot.stop()

In [62]:
encoding = "UTF-8"
forward = True
while forward:
    result = subprocess.run(["iwlist", "wlan0", "scan"], stdout=PIPE, stderr=PIPE)
    prediction = make_prediction(str(result.stdout, encoding), df)
    prediction[0] *= 12
    print(prediction[0])
    time.sleep(0.1)
    # Checkpoint set as a ~2.5 square foot area representing an "okay" area to turn in
    if (prediction[0][0] < 5.55 and prediction[0][0] > 3 and prediction[0][1] < 5.46 and prediction[0][1] > 3):
        robot.stop()
        time.sleep(2)
        robot.right(0.32)
        time.sleep(0.25)
        robot.stop()
        print("NOW!")
        forward = False
    robot.forward(0.4)
    time.sleep(0.9)
    robot.stop()
    robot.right(0.19)
    time.sleep(0.05)
    robot.stop()
time.sleep(0.3)
    # Proof of concept. After we've turned the corner, go straight
while True:
    robot.forward(0.4)
    time.sleep(0.9)
    robot.stop()
    robot.right(0.15)
    time.sleep(0.06)
    robot.stop()
    time.sleep(0.5)
    
# Note that there is Keyboard Interrupt here, because otherwise the robot would not have stopped.

[9.195884  4.1448765]
[8.789195  4.0180097]
[7.826026  4.4706516]
[5.00171  1.983942]
[7.156274  4.2299643]
[5.950962  5.7142463]
[4.868839  3.4932928]
NOW!


KeyboardInterrupt: 

In [55]:
# robot.forward(0.3)
# time.sleep(1)
robot.stop()

In [53]:
#Testing
while True:
    robot.forward(0.4)
    time.sleep(0.9)
    robot.stop()
    robot.right(0.16)
    time.sleep(0.06)
    robot.stop()
    time.sleep(0.5)


KeyboardInterrupt: 